In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
import time
from tqdm import tqdm, trange
import re

In [27]:
path_to_file = 'corpus.txt'
corpus = open(path_to_file, 'rb').read().decode(encoding='utf-8')

In [28]:
corpus = re.sub(r"[^a-zA-Z0-9 .,\[\]\(\)\n\']", "",corpus)

In [29]:
# The unique characters in the file
vocab = sorted(set(corpus))
print(f'{len(vocab)} unique characters')
print(f'Length of text: {len(corpus)} characters')

70 unique characters
Length of text: 3211598 characters


In [30]:
device = tf.device('device:GPU:0') 

chars = tf.strings.unicode_split(vocab, input_encoding='UTF-8')


In [31]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [32]:
vocab

[' ',
 "'",
 '(',
 ')',
 ',',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [33]:
ids = ids_from_chars(chars)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)
chars = chars_from_ids(ids)

In [34]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [35]:
all_ids = ids_from_chars(tf.strings.unicode_split(corpus, 'UTF-8'))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [36]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"'JERRY Do you know what this is all about Do you know, why were here To be out, this is out...and ou"
Target: b'JERRY Do you know what this is all about Do you know, why were here To be out, this is out...and out'


In [37]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


In [38]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [39]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [40]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [41]:
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [42]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 71) # (batch_size, sequence_length, vocab_size)


In [43]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [44]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'EORGE I dont know, what Eight thousand. Its a Hyundai. Get out of here. I told you not to sell. Simo'

Next Char Predictions:
 b'0IbLL56iL]ZQVIm47GRdy(U0,ml.ee5[UNK]s]T.XjAztcQ(A6n[UNK]tkYOlYKgiANB0qWUDFMicM]p8(71NEcWuxtVm[UNK]c ji04mIUSZN7J'


In [45]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 71)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.2627363, shape=(), dtype=float32)


In [46]:
tf.exp(example_batch_mean_loss).numpy()

71.004

In [47]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(), loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './seinfeld_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [48]:
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [49]:
EPOCHS = 5
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/5


2023-01-03 16:19:55.409302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:19:55.997844: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:19:56.048992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/496 [==============================] - 78s 156ms/step - loss: nan
Epoch 2/5
496/496 [==============================] - 77s 155ms/step - loss: nan
Epoch 3/5
496/496 [==============================] - 77s 154ms/step - loss: nan
Epoch 4/5
496/496 [==============================] - 77s 153ms/step - loss: nan
Epoch 5/5
496/496 [==============================] - 76s 151ms/step - loss: nan


In [50]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)
    print(skip_ids)

  @tf.function
  def generate_one_step(self, inputs, states=None,mask=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    print(predicted_logits)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask
    print(predicted_logits)

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    print(predicted_ids)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)
    print(predicted_chars)
    print(states)
    # Return the characters and model state.
    return predicted_chars, states

In [51]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

tf.Tensor([[0]], shape=(1, 1), dtype=int64)


In [52]:
one_step_model.generate_one_step(tf.constant(['JERRY: ']))

Tensor("my_model_1/dense_1/BiasAdd:0", shape=(1, None, 71), dtype=float32)
Tensor("add:0", shape=(1, 71), dtype=float32)
Tensor("Squeeze:0", shape=(1,), dtype=int64)
Tensor("string_lookup_3/None_Lookup/LookupTableFindV2:0", shape=(1,), dtype=string)
Tensor("my_model_1/gru_1/PartitionedCall:2", shape=(1, 1024), dtype=float32)


2023-01-03 16:26:19.846323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:26:19.935468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[UNK]'], dtype=object)>,
 <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=array([[nan, nan, nan, ..., nan, nan, nan]], dtype=float32)>)

In [53]:
start = time.time()
states = None
next_char = tf.constant(['JERRY:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Tensor("my_model_1/dense_1/BiasAdd:0", shape=(1, None, 71), dtype=float32)
Tensor("add:0", shape=(1, 71), dtype=float32)
Tensor("Squeeze:0", shape=(1,), dtype=int64)
Tensor("string_lookup_3/None_Lookup/LookupTableFindV2:0", shape=(1,), dtype=string)
Tensor("my_model_1/gru_1/PartitionedCall:2", shape=(1, 1024), dtype=float32)


2023-01-03 16:26:20.397754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:26:20.453338: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


JERRY:[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK

In [54]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [55]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [56]:
model.compile(optimizer = tf.keras.optimizers.legacy.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [57]:
model.fit(dataset, epochs=100)

Epoch 1/100


2023-01-03 16:26:29.917777: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:26:30.510323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-01-03 16:26:30.562116: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 2/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 3/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 4/100
496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 5/100
496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 6/100
496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 7/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 8/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 9/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 10/100
496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 11/100
496/496 [==============================] - 76s 152ms/step - loss: nan
Epoch 12/100
496/496 [==============================] - 76s 151ms/step - loss: nan
Epoch 13/100
496/496 [===

KeyboardInterrupt: 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['JERRY:'])
result = [next_char]

for n in range(10000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

JERRY:[UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK][UNK

In [ ]:
one_step_model.generate_one_step(tf.constant(['JERRY:']))

(<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'[UNK]'], dtype=object)>,
 <tf.Tensor: shape=(1, 1024), dtype=float32, numpy=array([[nan, nan, nan, ..., nan, nan, nan]], dtype=float32)>)